# Imports

In [ ]:
%matplotlib notebook

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from bgcellmodels.models.STN.GilliesWillshaw.optimize.bpop_optimize_stn import *

from bgcellmodels.common import logutils
logutils.setLogLevel('quiet', ['marasco', 'folding', 'redops'])

# Make Model

## Model Type

In [ ]:
# Make cell model

# SETPARAM: full/reduced model
model_type = StnModel.Gillies2005
# model_type = StnModel.Gillies_FoldBush_Tapered # 'full', 'reduced', StnModel.Gillies_FoldBush_Tapered

cellmodel = stn_models.make_cell_model(
                        model_type,
                        num_passes=7)

## Model Params

In [ ]:
# SETPARAM: Add model parameters
# params_biophys = stn_params.dend_gbar_dist_params
# cellmodel.add_params(params_biophys)

In [ ]:
# UNCOMMENT: Inspect model
# sim = ephys.simulators.NrnSimulator(dt=0.025, cvode_active=False)
# cellmodel.instantiate(sim=sim)
# from neuron import gui

# Run Protocols

List Protocols

In [ ]:
# SETPARAM: protocol
all_protos = [k for k,v in PROTOCOL_WRAPPERS.items() if v is not None]
# proto = StimProtocol.PRC_SYN_EXC_DIST
# proto = StimProtocol.PRC_SYN_EXC_PROX
# proto = StimProtocol.PRC_SYN_INH_PROX

print(all_protos)

## Spontaneous

In [ ]:
# Run protocol
test_protocol(StimProtocol.SPONTANEOUS, cellmodel, export_locals=True)

## Plateau

In [ ]:
# Run protocol
test_protocol(StimProtocol.CLAMP_PLATEAU, cellmodel, export_locals=True)

## PRC Protocols

In [ ]:
proto = StimProtocol.PRC_SYN_EXC_PROX
test_protocol(proto, cellmodel, export_locals=True)

In [ ]:
proto = StimProtocol.PRC_SYN_EXC_DIST
test_protocol(proto, cellmodel, export_locals=True)

In [ ]:
proto = StimProtocol.PRC_SYN_INH_PROX
test_protocol(proto, cellmodel, export_locals=True)

### ARTICLE figs

In [ ]:
from bgcellmodels.common import plotutils
from bgcellmodels.extensions.bluepyopt.efeatures_spiketrain import debug_data

# Tweak figure
fig, ax = debug_data

In [ ]:
ax.grid(True)
ax.set_xlim((-.01, 1.3))
ax.set_ylim((-0.3, 0.4))
ax.set_xlabel('stimulus phase (0-1)')
ax.set_ylabel('spike advance (normalized to 1)')
fig.show()

plotutils.save_figure(
    'stn-full_{}'.format(str(proto)[13:]), fig=fig, timestamp=True,
    dir='/home/luye/cloudstore_m/notes/publications_lkmn/ARTICLE_reduction/figures')

# Save Responses

In [ ]:
saved_protos = [k for k,v in PROTOCOL_WRAPPERS.items() if v is not None]
saved_protos = [k for k,v in PROTOCOL_WRAPPERS.items() if 'PRC' in str(k)]
# saved_protos = [StimProtocol.PRC_SYN_EXC_PROX]
print(saved_protos)

In [ ]:
save_protocol_responses(saved_protos)

# Save Notebook

In [ ]:
# If not automatically executed: save results here
from datetime import datetime
import os.path

nb_name = 'test_PRC_protocols'
thisfile = nb_name + '.ipynb'
version = str(cellmodel.model_type) + '-gbar-dist-orig'

outsuffix = '_model-' + version
out_dir = '/home/luye/cloudstore_m/simdata/Gillies2005_reduced'
outbase = nb_name + outsuffix + '_' + datetime.now().strftime('%Y.%m.%d-%H.%M.%S')
outfile_html = outbase + '.html'
outfile_full_html = os.path.join(out_dir, outfile_html)
outfile_full_nb = os.path.join(out_dir, outbase + '.ipynb')

# Export using built-in template:
#!jupyter nbconvert --to html --template=full --output-dir=$outputs $thisfile
# NOTE: template comes from ToC2 Notebook Extension
# NOTE: the class <tocskip> for headers messes up the export -> do not use
!jupyter nbconvert $thisfile --to html --template=toc2 --output-dir=$out_dir --output $outfile_html
# replace title
subs_regex = "'1,6s/<title>{}/<title>{}/g'".format(nb_name, version)
%system sed -i $subs_regex $outfile_full_html